# 패키지 임포트

In [1]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
import os
import string
import wordninja
import re
import json
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Attention
from keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from transformer_build import  *
from data_preprocessing import *
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

# 데이터 로드

In [2]:
data=[]
with open('C:/Users/user/Desktop/bilm-tf-master/textdataset/News_Category_Dataset_v3.json', 'r') as f:
    for line in f:
        content = json.loads(line)
        data.append(content)
df = pd.DataFrame(data=data)

In [3]:
textcuboid=np.load('./1-Channel textcuboid_nc(bert).npy')
textcuboid_test=np.load('./1-Channel textcuboid_test_nc(bert).npy')

# 전처리

In [4]:
df = df[~df['short_description'].apply(lambda x: len(x)==0)]

#중복되는 카테고리 통합
df["category"] = df.category.replace(
    {
        "THE WORLDPOST": "WORLD NEWS",
        "WORLDPOST": "WORLD NEWS",
        "ARTS": "ARTS & CULTURE",
        "CULTURE & ARTS": "ARTS & CULTURE",
        "HEALTHY LIVING": "WELLNESS",
        "QUEER VOICES": "VOICES",
        "LATINO VOICES": "VOICES",
        "BLACK VOICES": "VOICES",
        "STYLE": "STYLE & BEATUY",
        "GREEN": "ENVIRONMENT",
        "TASTE": "FOOD & DRINK",
        "MONEY": "BUSINESS",
        "PARENTING": "PARENTS"
    }
)

df = df[['headline', 'short_description','category']]
df['input_data']= df.apply(lambda x: str(x['headline']) + str(x['short_description']), axis=1)

df['clean_text']=df['input_data'].apply(clean_text)

class0=list(df[df['category'] == 'POLITICS'].sample(1000, random_state=1)['clean_text'])
class1=list(df[df['category'] == 'WELLNESS'].sample(1000, random_state=1)['clean_text'])
class2=list(df[df['category'] == 'ENTERTAINMENT'].sample(1000, random_state=1)['clean_text'])
class3=list(df[df['category'] == 'PARENTS'].sample(1000, random_state=1)['clean_text'])
class4=list(df[df['category'] == 'VOICES'].sample(1000, random_state=1)['clean_text'])
class5=list(df[df['category'] == 'STYLE & BEAUTY'].sample(1000, random_state=1)['clean_text'])
class6=list(df[df['category'] == 'TRAVEL'].sample(1000, random_state=1)['clean_text'])
class7=list(df[df['category'] == 'FOOD & DRINK'].sample(1000, random_state=1)['clean_text'])
class8=list(df[df['category'] == 'WORLD NEWS'].sample(1000, random_state=1)['clean_text'])
class9=list(df[df['category'] == 'BUSINESS'].sample(1000, random_state=1)['clean_text'])

X_list=class0+class1+class2+class3+class4+class5+class6+class7+class8+class9

y=[]
for i in range(10):
    y+=[i]*1000
    
clr_x_data=[]
pattern = '[^a-z ]'
for sen in X_list:
    clr_x_data.append(re.sub(pattern, ' ', sen))
    
X_list=[]
for sen in clr_x_data:
    X_list.append(' '.join(sen.split()))
    
train_idx=[]
for i in range(10):
    train_idx+=[j+i*1000 for j in range(800)]

test_idx=[]
for i in range(10):
    test_idx+=[j+800+i*1000 for j in range(200)]
    
x_train=[]
y_train=[]
for i in train_idx:
    x_train.append(X_list[i])
    y_train.append(y[i])

x_test=[]
y_test=[]
for i in test_idx:
    x_test.append(X_list[i])
    y_test.append(y[i])

In [5]:
sos_x_train=[]
sos_x_test=[]
for sen in x_train:
    sos_x_train.append('<sos> '+sen)
for sen in x_test:
    sos_x_test.append('<sos> '+sen)

all_txt=sos_x_train+sos_x_test

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_txt)

vocab_size =len(tokenizer.word_index)+1 #1을 더해야 에러가 안터짐 토큰 영향으로 보임

x_train_encoded = tokenizer.texts_to_sequences(sos_x_train)
x_test_encoded = tokenizer.texts_to_sequences(sos_x_test)

max_len = 165

xtext_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_encoded, maxlen=max_len)
xtext_test = tf.keras.preprocessing.sequence.pad_sequences(x_test_encoded, maxlen=max_len)

In [6]:
tmp = [[x,y,z] for x, y, z in zip(xtext_train, textcuboid, y_train)]
random.shuffle(tmp)
xtext_train = [n[0] for n in tmp]
textcuboid = [n[1] for n in tmp]
y_train = [n[2] for n in tmp]
xtext_train=np.array(xtext_train)
textcuboid=np.array(textcuboid)
y_train=np.array(y_train)

In [7]:
textcuboid_train=textcuboid[1000:]
text_train=xtext_train[1000:]
textcuboid__val=textcuboid[:1000]
text_val=xtext_train[:1000]
y_val=y_train[:1000]
y_train=y_train[1000:]

text_test=xtext_test
y_test=np.array(y_test)

# 모델 빌드

In [8]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_{epoch}.h5", save_best_only=False, period=5
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
]

In [9]:
embedding_dim = 768  # 각 단어의 임베딩 벡터의 차원
num_heads = 1  # 어텐션 헤드의 수
dff = 64  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기
num_transformer_blocks = 1 #트랜스포머 블록 적층

#BERT channel의 학습
BERT_input = Input(shape=(68, 768))
conv1 = Conv1D(1024, 1, padding='valid', activation='relu')(BERT_input)
pooling = GlobalMaxPooling1D()(conv1)
# pooling = Dropout(0.5)(pooling)
BERT_x = Dense(256, activation='relu')(pooling)
BERT_attention_layer = Attention()
BERT_a=BERT_attention_layer([BERT_x,BERT_x])

#Sequence channel의 학습
Text_inputs = Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
Text_x = embedding_layer(Text_inputs)

for _ in range(num_transformer_blocks):
    transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
    Text_x = transformer_block(Text_x)
    
Text_x = tf.keras.layers.GlobalAveragePooling1D()(Text_x)
# Text_x = tf.keras.layers.Dropout(0.5)(Text_x)
Text_x = tf.keras.layers.Dense(256, activation="relu")(Text_x)
Text_attention_layer = Attention()
Text_a=Text_attention_layer([Text_x,Text_x])

#결합 후 분류
x = layers.concatenate([BERT_x,BERT_a, Text_x,Text_a], axis=-1)
# x = layers.concatenate([BERT_x, Text_x], axis=-1)
x = Dropout(0.5)(x)
output_layer = Dense(10, activation='softmax')(x)

model = Model(inputs=[BERT_input, Text_inputs], outputs=output_layer)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 165)]        0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 68, 768)]    0           []                               
                                                                                                  
 token_and_position_embedding (  (None, 165, 768)    26670336    ['input_2[0][0]']                
 TokenAndPositionEmbedding)                                                                       
                                                                                                  
 conv1d (Conv1D)                (None, 68, 1024)     787456      ['input_1[0][0]']            

In [ ]:
history = model.fit([textcuboid_train,text_train], y_train,callbacks=callbacks, epochs=50,batch_size=256,validation_data=([textcuboid__val,text_val], y_val))

Epoch 1/50
28/28 [==============================] - 4s 84ms/step - loss: 3.8457 - accuracy: 0.1497 - val_loss: 1.9956 - val_accuracy: 0.2840 - lr: 0.0010
Epoch 2/50
23/28 [=======================>......] - ETA: 0s - loss: 2.0067 - accuracy: 0.2831

In [15]:
model.evaluate([textcuboid_test,text_test],y_test)

63/63 [==============================] - 1s 19ms/step - loss: 0.5905 - accuracy: 0.8210


[0.590481698513031, 0.8209999799728394]

In [27]:
from keras.models import load_model
custom_objects = {"TokenAndPositionEmbedding": TokenAndPositionEmbedding, "TransformerBlock": TransformerBlock}
model = load_model('best_model_45.h5', custom_objects=custom_objects)
model.evaluate([textcuboid_test,text_test],y_test)

63/63 [==============================] - 2s 21ms/step - loss: 0.6092 - accuracy: 0.8220


[0.6092294454574585, 0.8220000267028809]